In [1]:
import numpy as np
import random
import pandas as pd
import sys
import networkx as nx
import matplotlib.pyplot as plt
import math as mt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
a = sys.path.append('../modules/')  # endereco das funcoes implementadas por voce!
import sphere, sample_random, aux_operators_array, Operators_array, plot_3D

In [2]:
population = {'xmax': 10.0,
                'xmin': 0.0,
                'ymax': 10.0,
                'ymin': 0.0,
                'zlim': 10.0,
                'z_min': 0.0,
                'n_dip': 1,
                'n_pop': 10,
                'homogeneo': True
                }

In [3]:
def sample_random_coordinated(xmax, xmin, ymax, ymin, zlim, z_min, n):
    sorted_x1, sorted_y1,sorted_z1 = (np.random.uniform(xmin, xmax, n),
                                      np.random.uniform(ymin, ymax, n),
                                      np.random.uniform(z_min, zlim, n))
    
    return sorted_x1, sorted_y1, sorted_z1


In [4]:
def create_population(xmax, xmin, ymax, ymin, zlim, z_min, n_dip, n_pop, homogeneo):
    """
    Função com o objetivo de criar uma população com n indivíduos randômicos, que estaram de acordo com os parâmetros
    escolhidos.

    :param xmax: O valor máximo da coordenada X.
    :param ymax: O valor máximo da coordenada Y.
    :param zlim: O valor máximo da coordenada Z.
    :param xmin: O valor minímo da coordenada X.
    :param ymin: O valor minímo da coordenada Y.
    :param z_min: O valor minímo da coordenada Z.
    :param n_pop: O número de indivíduos desejados na população.
    :param n_dip: O número de dipolos desejados para cada indivíduo.
    :param inclmax: Valor máximo da inclianção magnética.
    :param inclmin: Valor mínimo da inclianção magnética.
    :param declmax: Valor máximo da inclianção magnética.
    :param declmin: Valor mínimo da declianção magnética.
    :param magmax: Valor máximo da magnetização.
    :param magmin: Valor mínimo da magnetização.
    :param homogeneo: True para valores de inclinação, declinação e magnetização iguais para as n dipolos.
                      False é a opção default, onde os valores de inclinação, declinação e magnetização é criada de
                      forma randômica.

    :return pop: Lista com n indivíduos/dipolos criados de forma randômica.
    """
    if n_pop >= 1:
        pop = []
        n_par = 3
        for j in range(n_pop):
            cood = np.zeros((n_dip, n_par))
            coodX, coodY, coodZ = sample_random_coordinated(xmax, xmin, ymax, ymin, zlim, z_min, n_dip)
            #print(type(coodX))
            #incl, decl, mag = sample_random.sample_random_mag(inclmax, inclmin, declmax, declmin, magmax, magmin, 1, homogeneo)
            for i in range(n_dip):
                cood[i][0], cood[i][1], cood[i][2] = coodX[i], coodY[i], coodZ[i]
            #cood[n_dip][0], cood[n_dip][1], cood[n_dip][2] = incl[0], decl[0], mag[0]
            pop += [cood]
        return pop
    else:
        return print('Por favor. Coloque o número de indivíduos maior ou igual a 1')

In [5]:
pop_inicial = create_population(**population)

In [6]:
print(pop_inicial)

[array([[6.38172363, 5.14947122, 7.87555518]]), array([[4.78489095, 7.26336674, 3.34169587]]), array([[8.58016944, 6.1596408 , 0.0887223 ]]), array([[9.93902776, 0.52342249, 7.24564862]]), array([[2.96479325, 5.40898807, 0.0285456 ]]), array([[7.10936691, 8.60075403, 3.6115268 ]]), array([[2.50480788, 2.29821917, 9.28029929]]), array([[7.28829528, 1.17705972, 8.21456143]]), array([[5.23198495, 7.2612895 , 2.80503471]]), array([[0.63211256, 6.44762396, 3.17027675]])]


## Uso dos grafos:

## To do: implementar a função de equidistância:
## calcular a média dos pessos do MST
## calcular a variancia dos pesos do MST
## $\theta = \sum_{i=1}^{M-1} [ d_{MST} - \overline{d}_{MST}]^2$

In [7]:
#Vetor de pesos:
pp = np.linspace(1, len(pop_inicial), len(pop_inicial))
ppm = int(sum(pp))
pesos = np.random.uniform(10.0, 50.0, ppm)
print(pesos)
print(len(pesos))

[29.17767677 20.72931261 16.91442428 39.50213984 35.43625981 31.68439473
 32.06615699 41.98111738 47.7411911  45.3950058  41.70720028 33.90094361
 23.59807316 19.84639085 14.87038721 49.47260265 29.00921069 24.92230699
 12.94591608 42.45934948 11.86546022 24.59882874 35.29959245 22.40720733
 42.74418066 26.72022087 21.88324271 30.78145305 48.90606269 33.56557759
 24.79980002 40.09788532 17.56630407 19.09334866 18.07665416 14.40065551
 12.83451005 42.47453533 47.67849432 34.20501219 26.81663942 28.71115604
 47.30874225 13.6762541  46.69914641 10.457899   44.28366284 10.28420354
 20.68664864 34.01273132 38.02641475 46.6393195  40.67383944 15.24930873
 36.80668451]
55


In [11]:
#Anexando os nodes e os edges ao grafo
def create_graph(pop_inicial, pesos):
    x , y, z = [], [], []
    for k in range(len(pop_inicial)):
        x_ = pop_inicial[k][0,0]
        y_ = pop_inicial[k][0,1]
        z_ = pop_inicial[k][0,2]
    
        x += [x_]
        y += [y_]
        z += [z_]
    x = np.array(x)
    print(x)
    y = np.array(y)
    z = np.array(z)

    grafos = nx.Graph()
    for i in range(len(x)):
        grafos.add_node(i ,pos=(x[i],y[i],z[i]))
        for j in range(len(x)):
            grafos.add_edge(i, j, weight=pesos[i])

    #Cálculo do MST
    TSG = nx.minimum_spanning_tree( grafos , algorithm='kruskal' )
    return grafos, TSG

In [12]:
def dist(x_coord,y_coord, z_coord):
    '''
    This function takes two vectors, x_coord and y_coord, and returns a matrix were the element in the ij position is the distance (considering the euclidian norm, ou l2 norm) betwen the point (x_coord[i],y_coord[i]) and (x_coord[j],y_coord[j]).
    
    Inputs:
    x_coord - numpy array 
    y_coord - numpy array
    
    Output:
    dl1 - numpy array - Matrix of distances
    '''
    
    #Stablishing the error condition
    tamx = np.shape(x_coord)[0]           
    tamy = np.shape(y_coord)[0]
    tamz = np.shape(z_coord)[0]
    if tamx != tamy or tamy != tamz or tamx != tamz:
        raise ValueError("All inputs must have same length!")
        
    #Calculating and savingn the distances
    #tam = ( math.factorial(tamx) )/( 2*(math.factorial(tamx-2)) ) #2 choises over 'tamx'(or 'tamy') posibilities
    distl2_matrix = np.zeros((tamx,tamy))
                                    
    for i in range(tamx):
        for j in range(tamx):
            distl2_matrix[i][j] = ( (x_coord[i] - x_coord[j])**2 + (y_coord[i] - y_coord[j])**2 + (z_coord[i] - z_coord[j])**2 )**(1/2)                                     
            
    return distl2_matrix

In [13]:
grafos, TSG = create_graph(pop_inicial, pesos)

[6.38172363 4.78489095 8.58016944 9.93902776 2.96479325 7.10936691
 2.50480788 7.28829528 5.23198495 0.63211256]


In [11]:
ed_tsg = nx.edges(TSG)
print(ed_tsg)
wt_tsg = TSG.edges.data('weight')
print(wt_tsg)

[(0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9)]
[(0, 9, 11.314096975852515), (1, 9, 11.314096975852515), (2, 9, 11.314096975852515), (3, 9, 11.314096975852515), (4, 9, 11.314096975852515), (5, 9, 11.314096975852515), (6, 9, 11.314096975852515), (7, 9, 11.314096975852515), (8, 9, 11.314096975852515)]


In [12]:
ed=nx.edges(grafos)
print(ed)
print(len(ed))

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 6), (6, 7), (6, 8), (6, 9), (7, 7), (7, 8), (7, 9), (8, 8), (8, 9), (9, 9)]
55


In [13]:
#Posições dos grafos
#pos = {}
#for i in range(len(pop_inicial)):
    #pos[i] = pop_inicial[i][0]
#print(pos)

In [14]:
dm1 = []
for (u, v, wt) in TSG.edges.data('weight'):
    dm1.append(wt)

print(dm1)

[11.314096975852515, 11.314096975852515, 11.314096975852515, 11.314096975852515, 11.314096975852515, 11.314096975852515, 11.314096975852515, 11.314096975852515, 11.314096975852515]


In [15]:
node_xyz = np.array([pos[v] for v in sorted(grafos)])
edge_xyz = np.array([(pos[u], pos[v]) for u, v in grafos.edges()])
# Create the 3D figure
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the nodes - alpha is scaled by "depth" automatically
ax.scatter(*node_xyz.T, s=100, ec="w")

# Plot the edges
for vizedge in edge_xyz:
    #print(vizedge)
    ax.plot(*vizedge.T, color="tab:gray")


def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(True)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    
    plt.title('Normal graph', fontsize = 30)

    ax.set_xticks(np.linspace(0, 10, 5))
    ax.set_yticks(np.linspace(0, 10, 5))
    ax.set_zticks(np.linspace(0, 10, 5))
    ax.tick_params(labelsize = 20)


_format_axes(ax)
fig.tight_layout()
plt.show()

NameError: name 'pos' is not defined

In [ ]:
node_xyz1 = np.array([pos[v] for v in sorted(TSG)])
edge_xyz1 = np.array([(pos[u], pos[v]) for u, v in TSG.edges()])
# Create the 3D figure
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the nodes - alpha is scaled by "depth" automatically
ax.scatter(*node_xyz1.T, s=100, ec="w")

# Plot the edges
for vizedge1 in edge_xyz1:
    #print(vizedge)
    ax.plot(*vizedge1.T, color="tab:gray")


def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(True)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    
    plt.title('Minimum spanning tree', fontsize = 30)

    ax.set_xticks(np.linspace(0, 10, 5))
    ax.set_yticks(np.linspace(0, 10, 5))
    ax.set_zticks(np.linspace(0, 10, 5))
    ax.tick_params(labelsize = 20)


_format_axes(ax)
fig.tight_layout()
plt.show()

## Manual Graph

In [ ]:
matrix = np.array([[10.0, 8.5, 2.0],
          [7.5, 4.2, 9.6],
          [4.7, 3.2, 5.6],
          [1.7, 7.8, 0.7]])

print(matrix)
print(len(matrix))
print(matrix[0,:])
m = matrix[0,:]
print(type(m))

In [ ]:
# coordenadas dos vertices (corners) do prisma, em metros:
x1,x2  = (0.0, 1.0) 
y1,y2  = (0.0, 1.0)
z1,z2  = (0.0, 1.0)   # z eh positivo para baixo!

In [ ]:
# Plot the main 3D source
figure = plt.figure(figsize=(10,10))

ax = figure.gca(projection = '3d')
#ax.set_title('Vertical dike model', size = 18)

#x1, x2, y1, y2, z1, z2 = fonte_crustal_mag[:6]
v = np.array([[x1, y1, z2], [x1, y2, z2], [x2, y2, z2], [x2, y1, z2], 
                 [x1, y1, z1], [x1, y2, z1], [x2, y2, z1], [x2, y1, z1]])
vert =  [[v[0],v[1],v[2],v[3]], 
         [v[0],v[1],v[5],v[4]], 
         [v[1],v[2],v[6],v[5]],
         [v[2],v[3],v[7],v[6]], 
         [v[3],v[0],v[4],v[7]], 
         [v[4],v[5],v[6],v[7]]]

fig01 = Poly3DCollection(vert, alpha = 0.75, linewidths = 0.75, edgecolors = 'k')
fig01.set_facecolor('chocolate')
ax.add_collection3d(fig01)
#individuo0 = ax.scatter(coodX1, coodY1, coodZ1, c=incl1, depthshade=True, cmap='jet', s = 200.0)

# Define the scale of the projection
x_scale = 1.2
y_scale = 1.2
z_scale = 1.
scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3] = 1.
def short_proj():
    return np.dot(Axes3D.get_proj(ax), scale)
ax.get_proj = short_proj

# Labels
ax.set_xlabel('North (m)', size = 25, labelpad = 30)
ax.set_ylabel('East (m)', size = 25, labelpad = 30)
ax.set_zlabel('Depth (m)', size = 25, labelpad = 30)
ax.set_xlim(-2.0, 2.0)
ax.set_ylim(-2.0, 2.0)
ax.set_zlim(-2.0, 2.0)
ax.set_xticks(np.arange(-2.0, 2.0, 4))
ax.set_yticks(np.linspace(-2.0, 2.0, 4))
ax.set_zticks(np.linspace(-2.0, 2.0, 4))
ax.tick_params(labelsize = 20, pad = 10)

# Visualization angle
ax.view_init(190, 130)

plt.tight_layout(True)
#plt.savefig('figs/dikemodel-r1.png', dpi = 300, bbox_inches = 'tight', transparent = True)
#plt.savefig('figs/dikemodel-r1.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
plt.show()

In [ ]:
#n! / (n-2)! (2)!
n_pesos = int(mt.factorial(len(v))/(mt.factorial(len(v)-2)*2))
print(len(v))
print(n_pesos)

pesos_igual = np.ones()

In [ ]:
grafos1, TSG1 = create_graph(v, pesos1)

In [ ]:
print(vert)
print('*************')
vert1 = np.array(vert)
print(vert1[0])
print(np.shape(vert1))

In [ ]:
def create_pop_manual(matrix):
    pop = []
    n_dip = 1
    n_par = 3
    for i in range(len(matrix)):
        cood = np.zeros((n_dip, n_par))
        cood[0,0], cood[0,1], cood[0,2] = matrix[i,0], matrix[i,1], matrix[i,2]
        pop += [cood]
    
    return pop

In [ ]:
pop_manual = create_pop_manual(matrix)

In [ ]:
print(pop_manual)
#print(pop_manual[1][])

In [ ]:
#Vetor de pesos:
pp1 = np.linspace(1, len(pop_manual), len(pop_manual))
ppm1 = int(sum(pp1))
pesos1 = np.random.uniform(10.0, 50.0, ppm1)
print(pesos1)
print(len(pesos1))

In [ ]:
grafos1, TSG1 = create_graph(pop_manual, pesos1)

In [ ]:
ed_tsg1 = nx.edges(TSG1)
print(ed_tsg1)

In [ ]:
node_xyz = np.array([pos[v] for v in sorted(grafos1)])
edge_xyz = np.array([(pos[u], pos[v]) for u, v in grafos1.edges()])
# Create the 3D figure
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the nodes - alpha is scaled by "depth" automatically
ax.scatter(*node_xyz.T, s=100, ec="w")

# Plot the edges
for vizedge in edge_xyz:
    #print(vizedge)
    ax.plot(*vizedge.T, color="tab:gray")


def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(True)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    
    plt.title('Normal graph', fontsize = 30)

    ax.set_xticks(np.linspace(0, 10, 5))
    ax.set_yticks(np.linspace(0, 10, 5))
    ax.set_zticks(np.linspace(0, 10, 5))
    ax.tick_params(labelsize = 20)


_format_axes(ax)
fig.tight_layout()
plt.show()

In [ ]:
node_xyz1 = np.array([pos[v] for v in sorted(TSG1)])
edge_xyz1 = np.array([(pos[u], pos[v]) for u, v in TSG1.edges()])
# Create the 3D figure
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection="3d")

# Plot the nodes - alpha is scaled by "depth" automatically
ax.scatter(*node_xyz1.T, s=100, ec="w")

# Plot the edges
for vizedge1 in edge_xyz1:
    #print(vizedge)
    ax.plot(*vizedge1.T, color="tab:gray")


def _format_axes(ax):
    """Visualization options for the 3D axes."""
    # Turn gridlines off
    ax.grid(True)
    # Suppress tick labels
    for dim in (ax.xaxis, ax.yaxis, ax.zaxis):
        dim.set_ticks([])
    # Set axes labels
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    
    plt.title('Minimum spanning tree', fontsize = 30)

    ax.set_xticks(np.linspace(0, 10, 5))
    ax.set_yticks(np.linspace(0, 10, 5))
    ax.set_zticks(np.linspace(0, 10, 5))
    ax.tick_params(labelsize = 20)


_format_axes(ax)
fig.tight_layout()
plt.show()